# Performance Branch Performance Tests - vortex_lattice_method

In [1]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

from numpy import sin, cos, tan, pi, dot, cross
from numpy.linalg import norm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from context import flyingcircus
from flyingcircus import geometry
from flyingcircus import basic_objects
from flyingcircus import mesh
from flyingcircus import vortex_lattice_method

## Data

In [4]:
# Geometric information
area = 20
aspect_ratio = 5
taper_ratio = 0.66
sweep_quarter_chord = 0
dihedral = 5
incidence = 0
torsion = 0
position = [0, 0, 0]

# Flight Conditions
alpha = 5
beta = 0
gamma = 0
attitude_vector = [alpha, beta, gamma]
altitude = 5000

true_airspeed = 100
flow_velocity_vector = geometry.velocity_vector(true_airspeed, alpha, beta, gamma)[:,0]
infinity_mult = 25

# Create wing object
wing = basic_objects.Wing(area, aspect_ratio, taper_ratio, sweep_quarter_chord, dihedral,
                              incidence, torsion, position)
infinity = infinity_mult * wing.wing_span

# Generate Mesh
n_semi_wingspam_panels = 10
n_chord_panels = 4
wingspam_discretization_type = "linear"
chord_discretization_type = "linear"

xx, yy, zz = mesh.generate_mesh(wing, n_semi_wingspam_panels, n_chord_panels,
                                    wingspam_discretization_type, chord_discretization_type)

# Create Panel Matrix
panel_matrix = mesh.generate_panel_matrix(xx, yy, zz, wing.wing_span)

## flatten

In [5]:
%%timeit
panel_vector = vortex_lattice_method.flatten(panel_matrix)

2.9 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## vortex_solver

In [6]:
panel_vector = vortex_lattice_method.flatten(panel_matrix)

In [8]:
%%timeit
gamma = vortex_lattice_method.vortex_solver(panel_vector, flow_velocity_vector,
                                                infinity_mult * wing.wing_span)

58.6 ms ± 4.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
gamma = vortex_lattice_method.vortex_solver(panel_vector, flow_velocity_vector,
                                                infinity_mult * wing.wing_span)

## vortex_solver components

### right_hand_side vector calculation

In [7]:
n_panels = len(panel_vector)
influence_coef_matrix = np.zeros((n_panels, n_panels))
right_hand_side_vector = np.zeros((n_panels, 1))

In [10]:
%%timeit
for i in range(n_panels):

    right_hand_side_vector[i][0] = dot(-flow_velocity_vector, panel_vector[i].n)

132 µs ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### influence_coef_matrix calculation

In [9]:
%%timeit
for i in range(n_panels):
    
    for j in range(n_panels):

            ind_vel, _ = panel_vector[j].hs_induced_velocity(panel_vector[i].col_point, 1)
            influence_coef_matrix[i][j] = dot(ind_vel, panel_vector[i].n)

50.4 ms ± 2.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%%timeit
for i in range(n_panels):
    
    for j in range(n_panels):

            ind_vel, _ = panel_vector[j].hs_induced_velocity(panel_vector[i].col_point, 1)

40.2 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### linear system solution

In [12]:
for i in range(n_panels):

    right_hand_side_vector[i][0] = dot(-flow_velocity_vector, panel_vector[i].n)
        
    for j in range(n_panels):

        ind_vel, _ = panel_vector[j].hs_induced_velocity(panel_vector[i].col_point, 1)
        influence_coef_matrix[i][j] = dot(ind_vel, panel_vector[i].n)

In [13]:
%%timeit
gamma, _ = sc.sparse.linalg.lgmres(influence_coef_matrix, right_hand_side_vector)

c:\users\jpmcost\.virtualenvs\masters_degree-zojzi3yy\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: scipy.sparse.linalg.lgmres called without specifying `atol`. The default value will change in the future. To preserve current behavior, set ``atol=tol``.
  """Entry point for launching an IPython kernel.


1.52 ms ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
